## Update `trips`
* cd rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env
* https://github.com/cal-itp/data-analyses/pull/1016
    * Keep source data + metrics tightly defined with GCS bucket organization.
    * vp_usable is source data for rt_vs_sched metrics, do not merge in schedule data until gtfs_digest report. Only bring in schedule_gtfs_dataset_key column in    
    * vp_usable + route_id-direction_id for trips also present in schedule. If not in schedule, fill it with route_id = Unknown and direction_id as Int64
    * Add function to concatenate trip file, enable us to put in 1 day or 7 days for aggregation
    * A single function for normalized metrics (percent, per min, etc)
    * A single function for aggregation (summing up numerator / denominator)
    
* https://github.com/cal-itp/data-analyses/issues/989

* Notes 2/6
    * GTFS digest creates four datasets: schedule, average speeds, segment speeds, and rt vs schedule
    * Currently, merging is challenging.
    * Time categories are not necessarily the same (peak/offpeak/all-day)
    * Want all datasets to merge on the same set of columns (schedule gtfs key, route id, dir id, service date, and time categories) because `shapes` are unstable.
    * `Route ID` has been stabilized by Tiffany 
    * Update work from `rt_v_scheduled.py` (steps already outlined in `scripts/route_aggregation.ipynb`)
        * Do steps up until row 339 when the % are calculated. 
        * Take away `speeds`.
        * Bring in schedule gtfs key, trip instance key, route id, direction id either at the beginning or the end using `helpers.import_scheduled_trips`
        * Coerce DIR ID to Int64, don't fill it in with 0. It's not 0, it's Nan
        * Save files with the analysis date at the end instead of the beginning.
        * Split off the workstream -> one for trip level and one for route level
            * Use the config.yml to save the trips and routes stuff into their own folder.
            * Routes:
                * For routes, the minutes/pings should be totalled up. Currently, just taking the average of an average isn't really accurate.
                * The route level should be able to take multiple days of data and concatenate so we can get metrics for a week/2 weeks/etc instead of for a single day. [Done here](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/scripts/average_speeds.py)
                * Add the route frequency as well?
           * Trips:
               * Do up to step 339 in `rt_v_scheduled.py`
               * Write a new generalized function to create all the % 
            
* Notes 2/13
    * Figure out how to set up Config file
    * Tiffany:
        * add_metrics looks good, just remove the coercing of percents to 0-100 to a separate function. I want everything from 0-1, and then before charting, scaled up to 0-100 all at once. Can you write a general         * function for this....all the chart display / cleaning functions should live in 1 script in segment_speed_utils.
        * Another tweak for a step somewhere before add_metrics. Certain columns can be coerced to be integers, like total_vp and vp_in_shape, just like how total_min_w_gtfs is an integer. Coerce all the ones that can be integers to be integers for your trip table, and this will save on the rounding step later.
        * Column naming: think about how you want to change the column names. total_pings_for_trip is not going to make sense once you aggregate, so maybe go with something more generic. Otherwise, you're going to be aggregating and renaming columns constantly. I would just rely on the other columns in the row to tell us whether it's per trip or per route , and the metrics all use generic names that are suitable for passing through aggregation functions. (edited) 

In [1]:
import dask.dataframe as dd
import pandas as pd
import yaml
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import RT_SCHED_GCS, SEGMENT_GCS
from shared_utils import portfolio_utils, rt_dates, rt_utils

In [2]:
# Times
import datetime

from loguru import logger

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
# analysis_date = rt_dates.DATES["dec2023"]

In [5]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [17]:
# rt_dates.DATES

### Routes - New Work

In [7]:
routes_df = pd.read_parquet("gs://calitp-analytics-data/data-analyses/rt_vs_schedule/vp_route_dir/route_direction_metrics/trip_2023_09_13_to_2023_10_11.parquet")

In [15]:
routes_df.sample()

,Time Period,Schedule Gtfs Dataset Key,Route Id,Direction Id,Total Min W Gtfs,Rt Service Min,Total Pings,Service Minutes,Total Vp,Vp In Shape,N Trips,Pings Per Min,Spatial Accuracy Pct,Rt W Gtfs Pct,Rt V Scheduled Time Pct
3433,peak,43d8d305ee692724a532f30ea63a1cbe,17,1,2212,2226,4354,1560.00,4354,3515,26,1.96,80.73,99.37,42.69


### Trips - New Work

In [18]:
trips_df = pd.read_parquet("gs://calitp-analytics-data/data-analyses/rt_vs_schedule/vp_trip/trip_metrics/trip_2023-09-13.parquet")

In [19]:
trips_df.sample()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings,total_min_w_gtfs,total_vp,vp_in_shape,route_id,direction_id,sched_rt_category,service_minutes,time_of_day,peak_offpeak,pings_per_min,spatial_accuracy_pct,rt_w_gtfs_pct,rt_v_scheduled_time_pct
33383,a068c9b5e54692d4f729dde66c36cdb8,dadb6ad81cea881a731c1be5978a481a,71,69,208,70,0,0,Unknown,<NA>,vp_only,NaN,Midday,offpeak,2.93,NaN,0.99,NaN


### Cleaning Function

In [10]:
pct_cols = [
    "rt_w_gtfs_pct",
    "rt_v_scheduled_time_pct",
    "spatial_accuracy_pct",
]

In [11]:
int_cols = [
    "rt_service_min",
    "service_minutes",
]

In [12]:
def clean_df(df: pd.DataFrame, pct_cols: list, int_cols: list) -> pd.DataFrame:
    for i in pct_cols:
        df[i] = df[i] * 100
    for i in int_cols:
        df[i] = df[i].fillna(0).round()

    df.columns = df.columns.str.replace("_", " ").str.strip().str.title()
    return df

In [13]:
routes_df2 = clean_df(routes_df, pct_cols, int_cols)

In [16]:
routes_df2['Time Period'].value_counts()

all_day    3174
peak       2981
offpeak    2729
Name: Time Period, dtype: int64

### Check time of day 
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/segment_speed_utils/segment_calcs.py#L135-L163

In [ ]:
analysis_date = "2023-10-11"

In [ ]:
trips = pd.read_parquet("gs://calitp-analytics-data/data-analyses/rt_vs_schedule/vp_trip/trip_metrics/trip_2023-10-11.parquet")

In [ ]:
trips.head()

In [ ]:
trips.peak_offpeak.unique()

In [ ]:
trips.time_of_day.value_counts()

In [ ]:
trips.peak_offpeak.value_counts()

In [ ]:
roll_singleday_route_dir_df = pd.read_parquet("gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_route_dir_2023-10-15.parquet")

In [ ]:
roll_singleday_route_dir_df.time_period.value_counts()

In [ ]:
roll_singleday_route_dir_df.sample()

In [ ]:
roll_singleday_speeds_df = pd.read_parquet("gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_trip_2023-10-15.parquet")

In [ ]:
roll_singleday_speeds_df.time_period.value_counts()

In [ ]:
len(roll_singleday_speeds_df), roll_singleday_speeds_df.trip_instance_key.nunique()

In [ ]:
roll_singleday_speeds_df.trip_instance_key.value_counts().head()

In [ ]:
roll_singleday_speeds_df.trip_instance_key.value_counts().describe()

In [ ]:
roll_singleday_speeds_df.sample()

In [ ]:
roll_singleday_speeds_df.time_period.value_counts()

In [ ]:
roll_singleday_speeds_df.loc[roll_singleday_speeds_df.trip_instance_key == "26a9df7b39cbcc34c8ea60d5f022cce1"]

In [ ]:
roll_singleday_speeds_df.loc[roll_singleday_speeds_df.trip_instance_key == "f552fb9336d5462a88bfa4a0f9e6f2ac"]

In [ ]:
roll_singleday_speeds_df.loc[roll_singleday_speeds_df.trip_instance_key == "04e042e8e5db03bf6d31b60455c895b6"]

 #### Fix Time of Day #1

In [ ]:
trips.columns

In [ ]:
trips2 = trips.drop(columns = [ 'peak_offpeak',
       'pings_per_min', 'spatial_accuracy_pct', 'rt_w_gtfs_pct',
       'rt_v_scheduled_time_pct'])

In [ ]:
trips2.head()

 #### Fix Time of Day #2

In [ ]:
def add_metrics(df: pd.DataFrame) -> pd.DataFrame:

    df["pings_per_min"] = df.total_pings / df.rt_service_min
    df["spatial_accuracy_pct"] = df.vp_in_shape / df.total_vp
    df["rt_w_gtfs_pct"] = df.total_min_w_gtfs / df.rt_service_min
    df["rt_v_scheduled_time_pct"] = df.rt_service_min / df.service_minutes - 1

    # Mask rt_triptime_w_gtfs_pct for any values above 100%
    df.rt_w_gtfs_pct = df.rt_w_gtfs_pct.mask(df.rt_w_gtfs_pct > 1, 1)

    return df

* yeah, so actually, i use the column peak_offpeak and i do not filter, but pass peak_offpeak in the grouping cols (route-direction-peak_offpeak). then i do it again without peak_offpeak in the group cols and rename peak_offpeak = all_day. then i concatenate.

In [ ]:
route_months = ["sep", "oct"]         

route_analysis_date_list = [
    rt_dates.DATES[f"{m}2023"] for m in route_months
]

In [ ]:
def concatenate_trip_segment_speeds(analysis_date_list: list) -> pd.DataFrame:
    """
    Concatenate the trip parquets together,
    whether it's for single day or multi-day averages.
    """
    TRIP_EXPORT = "vp_trip/trip_metrics"
    df = pd.concat(
        [
            pd.read_parquet(
                f"{RT_SCHED_GCS}{TRIP_EXPORT}/trip_{analysis_date}.parquet"
            ).assign(service_date=pd.to_datetime(analysis_date))
            for analysis_date in analysis_date_list
        ],
        axis=0,
        ignore_index=True,
    )
    return df


In [ ]:
def route_metrics(analysis_date_list: list) -> pd.DataFrame:
  
    df = concatenate_trip_segment_speeds(analysis_date_list)
    
    # Delete out trip generated metrics
    del_cols = [
        "pings_per_min",
        "spatial_accuracy_pct",
        "rt_w_gtfs_pct",
        "rt_v_scheduled_time_pct",
    ]

    df = df.drop(columns=del_cols)

    # Add weighted metrics
    sum_cols = [
        "total_min_w_gtfs",
        "rt_service_min",
        "total_pings",
        "service_minutes",
        "total_vp",
        "vp_in_shape",
    ]

    count_cols = ["trip_instance_key"]

    all_day_groups = ["schedule_gtfs_dataset_key",
                  "route_id",
                  "direction_id",
                  ]
    
    all_day_df = (
        df.groupby(all_day_groups)
        .agg({**{e: "sum" for e in sum_cols}, **{e: "count" for e in count_cols}})
        .reset_index()
    )

    all_day_df = all_day_df.rename(columns={"trip_instance_key": "n_trips"})
    all_day_df =  add_metrics(all_day_df)
    all_day_df['time_period'] = "all_day"
    
    peak_groups = ["peak_offpeak"] + all_day_groups
    peak_df = (
        df.groupby(peak_groups)
        .agg({**{e: "sum" for e in sum_cols}, **{e: "count" for e in count_cols}})
        .reset_index()
    )

    peak_df = peak_df.rename(columns={"trip_instance_key": "n_trips",'peak_offpeak':'time_period'})
    peak_df =  add_metrics(peak_df)
    
    final_df = pd.concat([peak_df, all_day_df])
    
    # Save
    #analysis_date_file = generate_date(analysis_date_list)
    #ROUTE_EXPORT = CONFIG_DICT["route_direction_metrics"]
    #df2.to_parquet(f"{RT_SCHED_GCS}{ROUTE_EXPORT}/trip_{analysis_date_file}.parquet")
    
    return final_df

In [ ]:
all_routes2 = route_metrics(route_analysis_date_list)

In [ ]:
all_routes2.head()

In [ ]:
all_routes2.time_period.value_counts()